In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import pandas as pd
import scanpy as sc 
import anndata as ann

In [5]:
# Load an example anndata object and pseudo-format it into a perturbation dataset
adata = sc.datasets.moignard15()

/home/hmaan/Documents/vae-vault/.venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
# Create some dummy perturbation columns
adata.obs['perturbation'] = np.random.choice(['A', 'B', 'C'], adata.shape[0])

# Create some dummy cell-type columns
adata.obs['cell_type'] = np.random.choice(['alpha', 'beta', 'gamma'], adata.shape[0])

In [7]:
import os 
import sys

sys.path.append("..")
sys.path.append("../../transforms/")

In [8]:
from splits import Scenarios, Scenario1
from method_transform import MethodTransform

/home/hmaan/scEvalsJam/scenarios/splits/examples/../../transforms/method_transform.py:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is "scgen":
/home/hmaan/scEvalsJam/scenarios/splits/examples/../../transforms/method_transform.py:40: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif method is "cpa":
/home/hmaan/scEvalsJam/scenarios/splits/examples/../../transforms/method_transform.py:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif method is "chemcpa":
/home/hmaan/scEvalsJam/scenarios/splits/examples/../../transforms/method_transform.py:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif method is "scpregan":
/home/hmaan/scEvalsJam/scenarios/splits/examples/../../transforms/method_transform.py:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif method is "gears":
/home/hmaan/scEvalsJam/scenarios/splits/examples/../../transforms/method_transform.py:48: SyntaxWarning: "is" with a literal. Did you mean "=="?

In [12]:
# Create a Method Transform object
method_transform = MethodTransform(
    adata=adata,
    gpu=False
)

# Process the data and return the processed anndata object
method_transform.process_data(
    method="scgen",
    celltype_key="cell_type",
    perturbation_key="perturbation",
)
processed_data=method_transform.return_anndata()

In [14]:
processed_data

AnnData object with n_obs × n_vars = 3934 × 42
    obs: 'exp_groups', 'perturbation', 'cell_type', 'batch_key', 'labels_key'
    uns: 'iroot', 'exp_groups_colors'

In [26]:
# Create a dummy 'dataset' variable and split the anndata based on this covariate 
dataset = np.random.choice(['norman', 'replogle'], processed_data.shape[0])
adata_train = processed_data[dataset == 'norman']
adata_test = processed_data[dataset == 'replogle']

In [28]:
# Read in scenario config
import yaml

scenario_config_path = "../configs/scenario_1_scgen.yaml"
try:
    with open(scenario_config_path, 'r') as file:
        scenario_config = yaml.safe_load(file)
    print(scenario_config)
except FileNotFoundError:
    print(f"Error: The file '{scenario_config_path}' was not found.")
except yaml.YAMLError as exc:
    print(f"Error parsing YAML file: {exc}")

{'method': 'scgen', 'test_size': 1, 'split_test': False, 'val_size': 0.3, 'scenario_name': 'Scenario 1 scGen', 'train_dataset': 'Test1', 'test_dataset': 'Test2', 'perturbation_key': 'perturbation', 'cell_type_key': 'cell_type', 'min_cells': 25, 'seed': 0}


In [29]:
scenario_config

{'method': 'scgen',
 'test_size': 1,
 'split_test': False,
 'val_size': 0.3,
 'scenario_name': 'Scenario 1 scGen',
 'train_dataset': 'Test1',
 'test_dataset': 'Test2',
 'perturbation_key': 'perturbation',
 'cell_type_key': 'cell_type',
 'min_cells': 25,
 'seed': 0}

In [30]:
# Initialize scenario class
scenario = Scenarios(
    scenario=Scenario1,
    adata_train=adata_train,
    adata_test=adata_test,
    scenario_config=scenario_config
)

In [31]:
scenario

In [61]:
# Perform train, test, val split using the splitter function in the Scenario class
# which is wrapped in the 'return_data' method of Scenarios 

# This will return three anndata objects - train val and test

train, val, test = scenario.return_data()

/home/hmaan/scEvalsJam/scenarios/splits/examples/../splits.py:72: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata_train.obs['split'] = 'train'
/home/hmaan/scEvalsJam/scenarios/splits/examples/../splits.py:73: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata_test.obs['split'] = 'test'
/home/hmaan/scEvalsJam/scenarios/splits/examples/../splits.py:96: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_train.obs['split'] = train_split_key
/home/hmaan/scEvalsJam/scenarios/splits/examples/../splits.py:97: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_test.obs['split'] = test_split_key


In [62]:
train

AnnData object with n_obs × n_vars = 1381 × 42
    obs: 'exp_groups', 'perturbation', 'cell_type', 'batch_key', 'labels_key', 'split', 'dataset'
    uns: 'iroot', 'exp_groups_colors'

In [63]:
train.obs["perturbation"].value_counts()

perturbation
B    475
C    466
A    440
Name: count, dtype: int64

In [64]:
train.obs["split"].value_counts()

split
train    1381
Name: count, dtype: int64

train.obs

In [65]:
val.obs["perturbation"].value_counts()

perturbation
B    204
C    200
A    189
Name: count, dtype: int64

In [66]:
val.obs["split"].value_counts()

split
validation    593
Name: count, dtype: int64

In [67]:
test.obs["perturbation"].value_counts()

perturbation
C    692
A    663
B    605
Name: count, dtype: int64

In [68]:
test.obs["split"].value_counts()

split
test    1960
Name: count, dtype: int64